<img src="https://github.com/hernancontigiani/ceia_memorias_especializacion/raw/master/Figures/logoFIUBA.jpg" width="500" align="center">


# Procesamiento de lenguaje natural
## Custom embeddings con Gensim



### Objetivo
El objetivo es utilizar documentos / corpus para crear embeddings de palabras basado en ese contexto. Se utilizará canciones de bandas para generar los embeddings, es decir, que los vectores tendrán la forma en función de como esa banda haya utilizado las palabras en sus canciones.

In [66]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import multiprocessing
from gensim.models import Word2Vec

### Datos
Utilizaremos como dataset canciones de bandas de habla inglesa.

In [3]:
# Descargar la carpeta de dataset
import os
import platform
if os.access('./songs_dataset', os.F_OK) is False:
    if os.access('songs_dataset.zip', os.F_OK) is False:
        if platform.system() == 'Windows':
            !curl https://raw.githubusercontent.com/FIUBA-Posgrado-Inteligencia-Artificial/procesamiento_lenguaje_natural/main/datasets/songs_dataset.zip -o songs_dataset.zip
        else:
            !wget songs_dataset.zip https://github.com/FIUBA-Posgrado-Inteligencia-Artificial/procesamiento_lenguaje_natural/raw/main/datasets/songs_dataset.zip
    !unzip -q songs_dataset.zip   
else:
    print("El dataset ya se encuentra descargado")

El dataset ya se encuentra descargado


In [4]:
# Posibles bandas
os.listdir("./songs_dataset/")

['disney.txt',
 'bob-marley.txt',
 'dickinson.txt',
 'bieber.txt',
 'bruno-mars.txt',
 'adele.txt',
 'amy-winehouse.txt',
 'radiohead.txt',
 'paul-simon.txt',
 'notorious_big.txt',
 'nicki-minaj.txt',
 'beatles.txt',
 'joni-mitchell.txt',
 'prince.txt',
 'johnny-cash.txt',
 'patti-smith.txt',
 'lil-wayne.txt',
 'al-green.txt',
 'michael-jackson.txt',
 'dr-seuss.txt',
 'blink-182.txt',
 'Kanye_West.txt',
 'alicia-keys.txt',
 'bob-dylan.txt',
 'janisjoplin.txt',
 'lady-gaga.txt',
 'dolly-parton.txt',
 'kanye-west.txt',
 'r-kelly.txt',
 'Lil_Wayne.txt',
 'drake.txt',
 'dj-khaled.txt',
 'leonard-cohen.txt',
 'lorde.txt',
 'notorious-big.txt',
 'bruce-springsteen.txt',
 'rihanna.txt',
 'eminem.txt',
 'jimi-hendrix.txt',
 'lin-manuel-miranda.txt',
 'kanye.txt',
 'missy-elliott.txt',
 'nickelback.txt',
 'ludacris.txt',
 'bjork.txt',
 'nursery_rhymes.txt',
 'cake.txt',
 'britney-spears.txt',
 'nirvana.txt']

In [5]:
# Armar el dataset utilizando salto de línea para separar las oraciones/docs
df = pd.read_csv('songs_dataset/beatles.txt', sep='/n', header=None)
df.head()

/tmp/ipykernel_14808/3849064916.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df = pd.read_csv('songs_dataset/beatles.txt', sep='/n', header=None)


,0
0,"Yesterday, all my troubles seemed so far away"
1,Now it looks as though they're here to stay
2,"Oh, I believe in yesterday Suddenly, I'm not h..."
3,There's a shadow hanging over me.
4,"Oh, yesterday came suddenly Why she had to go ..."


In [6]:
print("Cantidad de documentos:", df.shape[0])

Cantidad de documentos: 1846


### 1 - Preprocesamiento

In [8]:
from keras.preprocessing.text import text_to_word_sequence

sentence_tokens = []
# Recorrer todas las filas y transformar las oraciones
# en una secuencia de palabras (esto podría realizarse con NLTK o spaCy también)
for _, row in df[:None].iterrows():
    sentence_tokens.append(text_to_word_sequence(row[0]))

2024-03-18 18:31:35.087014: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-18 18:31:35.236175: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-18 18:31:35.237228: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-18 18:31:36.069095: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [9]:
# Demos un vistazo
sentence_tokens[:2]

[['yesterday', 'all', 'my', 'troubles', 'seemed', 'so', 'far', 'away'],
 ['now', 'it', 'looks', 'as', 'though', "they're", 'here', 'to', 'stay']]

### 2 - Crear los vectores (word2vec)

In [67]:
from gensim.models.callbacks import CallbackAny2Vec
# Durante el entrenamiento gensim por defecto no informa el "loss" en cada época
# Sobrecargamos el callback para poder tener esta información
class callback(CallbackAny2Vec):
    """
    Callback to print loss after each epoch
    """
    def __init__(self):
        self.epoch = 0

    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        if self.epoch == 0:
            print('Loss after epoch {}: {}'.format(self.epoch, loss))
        else:
            print('Loss after epoch {}: {}'.format(self.epoch, loss- self.loss_previous_step))
        self.epoch += 1
        self.loss_previous_step = loss

In [13]:
# Crearmos el modelo generador de vectores
# En este caso utilizaremos la estructura modelo Skipgram
w2v_model = Word2Vec(min_count=5,    # frecuencia mínima de palabra para incluirla en el vocabulario
                     window=2,       # cant de palabras antes y desp de la predicha
                     vector_size=300,       # dimensionalidad de los vectores 
                     negative=20,    # cantidad de negative samples... 0 es no se usa
                     workers=1,      # si tienen más cores pueden cambiar este valor
                     sg=1)           # modelo 0:CBOW  1:skipgram

In [14]:
# Obtener el vocabulario con los tokens
w2v_model.build_vocab(sentence_tokens)

In [15]:
# Cantidad de filas/docs encontradas en el corpus
print("Cantidad de docs en el corpus:", w2v_model.corpus_count)

Cantidad de docs en el corpus: 1846


In [16]:
# Cantidad de words encontradas en el corpus
print("Cantidad de words distintas en el corpus:", len(w2v_model.wv.index_to_key))

Cantidad de words distintas en el corpus: 445


### 3 - Entrenar embeddings

In [17]:
# Entrenamos el modelo generador de vectores
# Utilizamos nuestro callback
w2v_model.train(sentence_tokens,
                 total_examples=w2v_model.corpus_count,
                 epochs=20,
                 compute_loss = True,
                 callbacks=[callback()]
                 )

Loss after epoch 0: 113208.15625
Loss after epoch 1: 65953.953125
Loss after epoch 2: 65934.265625
Loss after epoch 3: 65711.96875
Loss after epoch 4: 63872.5
Loss after epoch 5: 64142.46875
Loss after epoch 6: 64058.8125
Loss after epoch 7: 64755.4375
Loss after epoch 8: 62585.5625
Loss after epoch 9: 60415.25
Loss after epoch 10: 59823.625
Loss after epoch 11: 58930.1875
Loss after epoch 12: 57726.4375
Loss after epoch 13: 56493.125
Loss after epoch 14: 55842.5625
Loss after epoch 15: 55862.5
Loss after epoch 16: 51695.8125
Loss after epoch 17: 49815.5
Loss after epoch 18: 49568.125
Loss after epoch 19: 48959.75


(156986, 287740)

### 4 - Ensayar

In [18]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["darling"], topn=10)

[('pretty', 0.9087147116661072),
 ('sleep', 0.8649678230285645),
 ('help', 0.8594305515289307),
 ('try', 0.8456019759178162),
 ('cry', 0.8387227654457092),
 ('little', 0.8273633718490601),
 ('not', 0.8181102871894836),
 ('seems', 0.8175203204154968),
 ('twist', 0.8154979348182678),
 ('peace', 0.812263011932373)]

In [19]:
# Palabras que MENOS se relacionan con...:
w2v_model.wv.most_similar(negative=["love"], topn=10)

[('shake', -0.21786975860595703),
 ('our', -0.24495810270309448),
 ('come', -0.2613270878791809),
 ('on', -0.26722386479377747),
 ('bang', -0.27329546213150024),
 ('five', -0.2741836607456207),
 ('six', -0.2774038314819336),
 ('baby', -0.2776353061199188),
 ('work', -0.2790372967720032),
 ('four', -0.2792743146419525)]

In [20]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["four"], topn=10)

[('five', 0.9786694049835205),
 ('three', 0.9761051535606384),
 ('six', 0.966891348361969),
 ('two', 0.9565720558166504),
 ('seven', 0.954232931137085),
 ('sixty', 0.9003373980522156),
 ('one', 0.816501259803772),
 ('us', 0.7762899398803711),
 ('crying', 0.772737979888916),
 ('strawberry', 0.7672452330589294)]

In [21]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["money"], topn=5)

[("can't", 0.9409592747688293),
 ('buy', 0.9399508237838745),
 ('much', 0.8996850848197937),
 ('hide', 0.8465201258659363),
 ('just', 0.8415144085884094)]

In [22]:
# Ensayar con una palabra que no está en el vocabulario:
w2v_model.wv.most_similar(negative=["diedaa"])

KeyError: "Key 'diedaa' not present in vocabulary"

In [23]:
# el método `get_vector` permite obtener los vectores:
vector_love = w2v_model.wv.get_vector("love")
print(vector_love)

[ 0.17736822  0.21460655 -0.05408018  0.06707036  0.13059464 -0.25713933
  0.00711611  0.26743424 -0.03100746  0.259963    0.21179457 -0.08465387
 -0.26272097  0.308386   -0.07934437 -0.13357355  0.2994755   0.18062147
 -0.13111673 -0.11857472  0.03468326  0.1125607   0.08478875  0.08090881
 -0.17622267  0.03122034 -0.08725598  0.23746666 -0.11507358 -0.29119617
 -0.03650665 -0.3322312   0.23262788  0.10366543 -0.18365337  0.08299578
  0.14315338 -0.19761457 -0.00757894  0.1119568  -0.10421463 -0.13083619
 -0.03264419 -0.04687435  0.11330808  0.10231452 -0.16202034 -0.10347028
  0.03659901 -0.08524423 -0.34872082 -0.02154255  0.25938764  0.16540806
  0.0310861   0.09505633  0.24452917 -0.09859891  0.18298703  0.0645447
  0.016967   -0.3259168  -0.04774074 -0.05418535 -0.04338397 -0.09137302
  0.00889874  0.11636022 -0.25662988  0.05426472  0.08915784  0.04541444
  0.20835714 -0.19101828  0.32210803  0.19337991  0.09080223 -0.00362429
 -0.18664253 -0.07053486 -0.1213743  -0.03477469 -0.

In [25]:
# el método `most_similar` también permite comparar a partir de vectores
w2v_model.wv.most_similar(vector_love)

[('love', 1.0),
 ('babe', 0.9132986068725586),
 ('someone', 0.8914992809295654),
 ('nothing', 0.8811879754066467),
 ('need', 0.8758756518363953),
 ("didn't", 0.8689683079719543),
 ("there's", 0.8557919859886169),
 ('you', 0.8447725772857666),
 ('feed', 0.8422104716300964),
 ('somebody', 0.8385539054870605)]

In [26]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["love"], topn=10)

[('babe', 0.9132986068725586),
 ('someone', 0.8914992809295654),
 ('nothing', 0.8811879754066467),
 ('need', 0.8758756518363953),
 ("didn't", 0.8689682483673096),
 ("there's", 0.8557919859886169),
 ('you', 0.8447725772857666),
 ('feed', 0.8422104716300964),
 ('somebody', 0.8385538458824158),
 ('hope', 0.8361446261405945)]

### 5 - Visualizar agrupación de vectores

In [68]:
from sklearn.decomposition import IncrementalPCA    
from sklearn.manifold import TSNE                   
import numpy as np                                  

def reduce_dimensions(model, num_dimensions = 2 ):
     
    vectors = np.asarray(model.wv.vectors)
    labels = np.asarray(model.wv.index_to_key)  

    tsne = TSNE(n_components=num_dimensions, random_state=0)
    vectors = tsne.fit_transform(vectors)

    return vectors, labels

In [29]:
# Graficar los embedddings en 2D
import plotly.graph_objects as go
import plotly.express as px

vecs, labels = reduce_dimensions(w2v_model)

MAX_WORDS=200
fig = px.scatter(x=vecs[:MAX_WORDS,0], y=vecs[:MAX_WORDS,1], text=labels[:MAX_WORDS])
#fig.show(renderer="colab") # esto para plotly en colab
fig.show()

In [30]:
# Graficar los embedddings en 3D

vecs, labels = reduce_dimensions(w2v_model,3)

fig = px.scatter_3d(x=vecs[:MAX_WORDS,0], y=vecs[:MAX_WORDS,1], z=vecs[:MAX_WORDS,2],text=labels[:MAX_WORDS])
fig.update_traces(marker_size = 2)
#fig.show(renderer="colab") # esto para plotly en colab
fig.show()

In [31]:
# También se pueden guardar los vectores y labels como tsv para graficar en
# http://projector.tensorflow.org/


vectors = np.asarray(w2v_model.wv.vectors)
labels = list(w2v_model.wv.index_to_key)

np.savetxt("vectors.tsv", vectors, delimiter="\t")

with open("labels.tsv", "w") as fp:
    for item in labels:
        fp.write("%s\n" % item)

### Alumno

- Crear sus propios vectores con Gensim basado en lo visto en clase con otro dataset.
- Probar términos de interés y explicar similitudes en el espacio de embeddings (sacar conclusiones entre palabras similitudes y diferencias).
- Graficarlos.
- Obtener conclusiones.

#### 1. Creación de vectores propios con Gensim utilizando un nuevo dataset

Se decidió emplear un dataset de noticias, para poder analizar fácilmente si las relaciones entre palabras obedecen a la realidad de las noticias que todos tenemos a disposición y conocemos. Se empleará un dataset de la BBC con noticias recientes.

fuente: https://www.kaggle.com/datasets/gpreda/bbc-news

Se subirá el archivo al repositorio, dado su pequeño tamaño y la imposibilidad de descargar desde Kaggle directamente sin establecer una sesión antes.

El archivo csv contiene dos columnas que son de nuestro interés, las columnas "title" y "description". Se generará un nuevo dataset concatenando ambas columnas.

In [69]:
import pandas as pd

df_full_bbc = pd.read_csv('bbc_news/bbc_news.csv')

# keep the two interesting columns
df_bbc_title = df_full_bbc['title']
df_bbc_description = df_full_bbc['description']

df_bbc = pd.DataFrame({'news': df_bbc_title + ' ' + df_bbc_description})

In [70]:
df_bbc.head()

,news
0,Ukraine: Angry Zelensky vows to punish Russian...
1,War in Ukraine: Taking cover in a town under a...
2,Ukraine war 'catastrophic for global food' One...
3,Manchester Arena bombing: Saffie Roussos's par...
4,Ukraine conflict: Oil price soars to highest l...


In [71]:
print("Cantidad de documentos: ", df_bbc.shape[0])

Cantidad de documentos:  30841


##### 1. Preprocesamiento

In [72]:
from keras.preprocessing.text import text_to_word_sequence

bbc_sentence_tokens = []
# Recorrer todas las filas y transformar las oraciones
# en una secuencia de palabras (esto podría realizarse con NLTK o spaCy también)
for _, row in df_bbc[:None].iterrows():
    bbc_sentence_tokens.append(text_to_word_sequence(row[0]))

In [73]:
# Demos un vistazo
bbc_sentence_tokens[:2]

[['ukraine',
  'angry',
  'zelensky',
  'vows',
  'to',
  'punish',
  'russian',
  'atrocities',
  'the',
  'ukrainian',
  'president',
  'says',
  'the',
  'country',
  'will',
  'not',
  'forgive',
  'or',
  'forget',
  'those',
  'who',
  'murder',
  'its',
  'civilians'],
 ['war',
  'in',
  'ukraine',
  'taking',
  'cover',
  'in',
  'a',
  'town',
  'under',
  'attack',
  'jeremy',
  'bowen',
  'was',
  'on',
  'the',
  'frontline',
  'in',
  'irpin',
  'as',
  'residents',
  'came',
  'under',
  'russian',
  'fire',
  'while',
  'trying',
  'to',
  'flee']]

##### 2. Creación de los vectores (word2vec)

Se empleará el callback declarado en el ejemplo anterior. A continuación se genera el modelo.

In [74]:
# Crearmos el modelo generador de vectores
# En este caso utilizaremos la estructura modelo Skipgram
bbc_w2v_model = Word2Vec(min_count=5,       # frecuencia mínima de palabra para incluirla en el vocabulario
                        window=2,           # cant de palabras antes y desp de la predicha
                        vector_size=300,    # dimensionalidad de los vectores 
                        negative=20,        # cantidad de negative samples... 0 es no se usa
                        workers=1,          # si tienen más cores pueden cambiar este valor
                        sg=1)               # modelo 0:CBOW  1:skipgram

In [75]:
# Obtener el vocabulario con los tokens
bbc_w2v_model.build_vocab(bbc_sentence_tokens)

In [76]:
# Cantidad de filas/docs encontradas en el corpus
print("Cantidad de docs en el corpus:", bbc_w2v_model.corpus_count)

Cantidad de docs en el corpus: 30841


In [77]:
# Cantidad de words encontradas en el corpus
print("Cantidad de words distintas en el corpus:", len(bbc_w2v_model.wv.index_to_key))

Cantidad de words distintas en el corpus: 12464


##### 3. Entrenamiento de embeddings

In [80]:
# Entrenamos el modelo generador de vectores
# Utilizamos nuestro callback
bbc_w2v_model.train(bbc_sentence_tokens,
                    total_examples=bbc_w2v_model.corpus_count,
                    epochs=20,
                    compute_loss = True,
                    callbacks=[callback()]
                    )

Loss after epoch 0: 697419.375
Loss after epoch 1: 606534.875
Loss after epoch 2: 584711.375
Loss after epoch 3: 559244.375
Loss after epoch 4: 490718.75
Loss after epoch 5: 536548.5
Loss after epoch 6: 479567.75
Loss after epoch 7: 455987.0
Loss after epoch 8: 484794.0
Loss after epoch 9: 437055.5
Loss after epoch 10: 518887.0
Loss after epoch 11: 472786.0
Loss after epoch 12: 464981.5
Loss after epoch 13: 463605.0
Loss after epoch 14: 461153.5
Loss after epoch 15: 457488.0
Loss after epoch 16: 432646.5
Loss after epoch 17: 372884.0
Loss after epoch 18: 373370.0
Loss after epoch 19: 445745.0


(13588225, 17213180)

#### 2. Prueba de términos de interés en el espacio de embeddings

A continuación se realizarán ensayos sobre determinados términos.

In [82]:
# Palabras que MÁS se relacionan con...:
bbc_w2v_model.wv.most_similar(positive=["ukraine"], topn=10)

[('war', 0.5154433846473694),
 ('liberate', 0.42660871148109436),
 ('defences', 0.422809898853302),
 ('melitopol', 0.4223330616950989),
 ('shelling', 0.41680940985679626),
 ('tigray', 0.41635191440582275),
 ('steelworks', 0.41454654932022095),
 ('kharkiv', 0.41412562131881714),
 ('olena', 0.41406890749931335),
 ("'russian", 0.4071860611438751)]

Se observa una muy buena relación entre "ukraine" y los términos más similares. Aparece "war" como término más similar, lo cual sabemos  es real, puesto que el país se encuentra actualmente en guerra con Rusia (término "russian" también aparece como similar). "melitopol", "kharkiv" son ciudades de Ucrania, y "olena" Zelenska es la primera dama de dicho país. "steelwork" y sobre todo "shelling" son términos asociados con la guerra.
"tigray" hace referencia a la guerra de Etiopía entre las autoridades regionales y el gobierno federal.

In [98]:
# Palabras que MENOS se relacionan con...:
bbc_w2v_model.wv.most_similar(negative=["peace"], topn=10)

[('luxury', 0.08041912317276001),
 ('swept', 0.0651436522603035),
 ('damaging', 0.05489848554134369),
 ('touring', 0.05391918122768402),
 ('caught', 0.05375427007675171),
 ('defends', 0.05148017033934593),
 ('deaths', 0.04874803498387337),
 ('officers', 0.0478413961827755),
 ('among', 0.04738779738545418),
 ('affected', 0.04504033550620079)]

En cuanto al análisis de los términos que menos se relacionan con "peace", podemos ver que hay palabras que claramente se diferencian simplemente por su significado, como ser "damaging", "deaths", "affected" y "officers". En cuanto al resto de los términos, la relación opuesta no termina de ser tan clara. Probablemente en el contexto de las noticias empleadas para el entrenamiento, tenga más sentido esa baja relación.

In [99]:
# Palabras que MÁS se relacionan con...:
bbc_w2v_model.wv.most_similar(positive=["hunger"], topn=10)

[('thirst', 0.40115973353385925),
 ('melitopol', 0.39056992530822754),
 ('palestinians', 0.3868604600429535),
 ('fatigue', 0.3854544162750244),
 ('cholera', 0.3854098320007324),
 ('starvation', 0.3724507689476013),
 ('prominent', 0.37125056982040405),
 ('ruptured', 0.36921489238739014),
 ('donkeys', 0.3670460283756256),
 ('starving', 0.3641813397407532)]

Nuevamente vemos una clara relación entre el término "hunger" y las palabras más similares. "thirst", "fatigue", "cholera", "starvation" y "starving" son palabras que claramente se relacionan, simplemente por su significado. "melitopol" es una ciudad en Ucrania, en el medio de la guerra, puede que haya problemas de abastecimiento en dicha ciudad. En cuanto a "palestinians", sabemos que los palestinos no están recibiendo la debida ayuda humanitaria, por lo que están viviendo una situación de hambruna. Para el resto de los términos, probablemente en el contexto de las noticias tenga mayor sentido la alta relación. No es tan clara a simple vista.

In [85]:
# Palabras que MÁS se relacionan con...:
bbc_w2v_model.wv.most_similar(positive=["strike"], topn=10)

[('strikes', 0.45282885432243347),
 ('walkout', 0.4338873326778412),
 ('neu', 0.40010377764701843),
 ("nurses'", 0.3977963626384735),
 ("miners'", 0.3953447937965393),
 ('ucu', 0.39414140582084656),
 ("doctors'", 0.3924266993999481),
 ("actors'", 0.38877981901168823),
 ('teachers’', 0.3820103406906128),
 ('aslef', 0.3806672990322113)]

Las términos que más se relacionan con "strike" tienen también una muy buena similaridad. Tanto "neu" que es la National Education Union (NEU), como el "ucu" que es la University and College Union (UCU) son entidades que representan los derechos de trabajadores de la educación. Luego, se relaciona fuertemente con profesiones; "nurses", "miners", "doctors", "actors" y "teachers", lo cual es por demás lógico. "aslef" es la trade union for train drivers, que también hace muchísimo sentido que tenga relación con "strike". "walkout" significa salir, que también ha de referirse a cuando un paro concluye.

In [86]:
# Palabras que MÁS se relacionan con...:
bbc_w2v_model.wv.most_similar(positive=["manchester"], topn=10)

[('united', 0.512407660484314),
 ("united's", 0.5002356171607971),
 ('ralf', 0.48592609167099),
 ('lisandro', 0.474539190530777),
 ('etihad', 0.46321868896484375),
 ('arsenal', 0.46205225586891174),
 ('rangnick', 0.4585292637348175),
 ('ilkay', 0.44900813698768616),
 ('£80m', 0.4463768005371094),
 ("rashford's", 0.4380016028881073)]

El término "manchester" está fuertemente relacionado con el fútbol. Vemos los términos "united", "united's", "etihad", "arsenal" por el club adversario, y también nombre de jugadores, "lisandro" (Martínez), "ilkay" (Gündoğan, ex Manchester City), (Marcus) "rashford's" y "ralf" "rangnick" un DT alemán, ex-asesor del Manchester United. "£80m" puede deberse a que ha aparecido en más de una noticia referente al Manchester United como valor de un pase importante.

In [87]:
# el método `get_vector` permite obtener los vectores:
bbc_vector_war = bbc_w2v_model.wv.get_vector("war")
print(bbc_vector_war)

[-0.22957268  0.17279886 -0.25996152 -0.27447563 -0.32720587 -0.3681116
  0.32035592 -0.02555706 -0.12288066  0.2812511   0.34995818  0.21692874
  0.20834771  0.28817853  0.39218482  0.12521784 -0.03791897 -0.3410448
  0.30562314 -0.17240627  0.09445775  0.24977037 -0.51704574  0.23517181
 -0.3733905   0.4534488   0.01246922  0.2036458  -0.20504987  0.00679487
  0.11213224 -0.635061   -0.13854812  0.5062624  -0.06803124 -0.10975668
  0.1381057  -0.02341348  0.02849276 -0.2587854   0.11028105 -0.2969246
 -0.25879818  0.1852365   0.32290792  0.3280002  -0.3014426   0.2522195
  0.40830478  0.1739144   0.5483339  -0.18622658  0.00804787  0.0403551
  0.32513928  0.2598468  -0.44844738  0.23241788  0.05730254 -0.07371601
 -0.10393019 -0.14750198 -0.11595561 -0.07195824  0.10890112 -0.21588199
  0.655882   -0.14612935 -0.6674625  -0.18093868  0.05891358 -0.17924641
 -0.25893402  0.13644218  0.11815779  0.47648275 -0.02360324 -0.23107898
  0.21909374  0.16192733 -0.07683867 -0.15919973 -0.1446

In [89]:
# el método `most_similar` también permite comparar a partir de vectores
bbc_w2v_model.wv.most_similar(bbc_vector_war)

[('war', 1.0),
 ('ukraine', 0.5154433846473694),
 ('klitschko', 0.41299930214881897),
 ('falklands', 0.41047409176826477),
 ('volodymyr', 0.40536877512931824),
 ('lavrov', 0.40242138504981995),
 ('tigray', 0.3986217975616455),
 ('ukraine’s', 0.39484021067619324),
 ('shelling', 0.39351963996887207),
 ('regiment', 0.3929310142993927)]

En este ejemplo se puede ver nuevamente que la relación es muy buena, simplemente con el análisis hecho anteriormente para el término "ukraine" se puede entender. Aparecen términos nuevos como (Vladimir) "klitschko", un ex-boxeador profesional ucraniano, "volodymyr" Zelenski, presidente de Ucrania y Serguei "lavrov", político y diplomático ruso. Probablemente "falklands" aparece en referencia a la guerra acontecida en 1982.

Los ensayos resultan muy satisfactorios. La relación entre los términos es bastante fuerte.

#### 3. Gráficos

A fin de visualizar la agrupación de vectores, se empleará la función reduce_dimensions(...) definida en el ejemplo.

In [90]:
# Graficar los embeddings en 2D
import plotly.graph_objects as go
import plotly.express as px

bbc_vecs, bbc_labels = reduce_dimensions(bbc_w2v_model)

MAX_WORDS=200
fig = px.scatter(x=bbc_vecs[:MAX_WORDS,0], y=bbc_vecs[:MAX_WORDS,1], text=bbc_labels[:MAX_WORDS])
# fig.show(renderer="colab") # esto para plotly en colab
fig.show()

In [91]:
# Graficar los embedddings en 3D

bbc_vecs, bbc_labels = reduce_dimensions(bbc_w2v_model,3)

fig = px.scatter_3d(x=bbc_vecs[:MAX_WORDS,0], y=bbc_vecs[:MAX_WORDS,1], z=bbc_vecs[:MAX_WORDS,2],text=bbc_labels[:MAX_WORDS])
fig.update_traces(marker_size = 2)
# fig.show(renderer="colab") # esto para plotly en colab
fig.show()

A continuación, se analizarán gráficamente algunos de los ejemplos expuestos en la sección "Prueba de términos de interés en el espacio de embeddings"

![alt text](image-2.png)

Aquí se puede ver de forma gráfica una fuerte relación entre los términos Ucrania, Rusia, Israel, Gaza y guerra. Tal como se expuso anteriormente, sabemos que estas relaciones son correctas, ya que estos países se encuentran en guerra actualmente.


Analicemos otro ejemplo:

![alt text](image-3.png)

En este caso, se puede observar una clara relación entre los términos Manchester, United, City, Chelsea, Liverpool, West y Premier League. Estos términos se relacionan fuertemente con el fútbol inglés, tal como se expuso en el ejemplo de similaridad con el término "manchester".

Cabe aclarar que no se ven exactamente los mismos términos que en los ejemplos analizados en la sección "Prueba de términos de interés en el espacio de embeddings". Esto se debe a la reducción de dimensiones y de puntos a la hora de realizar los gráficos. De todas formas, los gráficos resultan también una herramienta valiosa para verificar que el nivel de relación entre los términos cercanos es igualmente muy bueno.

#### 4. Conclusiones

En resumen, los resultados obtenidos demuestran la eficacia y la capacidad del modelo Word2Vec de Gensim para capturar relaciones semánticas entre palabras en un contexto específico. Se ha podido observar de forma clara que las relaciones entre las palabras similares para el contexto de noticias de la BBC son muy buenas.

Se concluye finalmente que el modelo Word2Vec resulta una herramienta efectiva para generar embeddings que representen una relación semántica correcta entre las palabras de un corpus determinado.